# This is your app

Let's get cranking

In [4]:
import { loadSync } from "jsr:@std/dotenv";
import { GoogleGenerativeAI } from "@google/generative-ai";
import { GoogleAIFileManager } from "@google/generative-ai/server";

loadSync({ envPath: "./.env" });

const apiKey = Deno.env.get("GEMINI_KEY")!;
const gemini = new GoogleGenerativeAI(apiKey).getGenerativeModel({ model: "gemini-2.0-flash" });
const fileManager = new GoogleAIFileManager(apiKey);

/**
 * Uploads the given file to Gemini.
 *
 * See https://ai.google.dev/gemini-api/docs/prompting_with_media
 */
async function uploadToGemini(path, mimeType) {
  const uploadResult = await fileManager.uploadFile(path, {
    mimeType,
    displayName: path,
  });
  const file = uploadResult.file;
  console.log(`Uploaded file ${file.displayName} as: ${file.name}`);
  return file;
}

{}

In [6]:
const generationConfig = {
  temperature: 1,
  topP: 0.95,
  topK: 40,
  maxOutputTokens: 8192 * 15 * 2,
  responseMimeType: "text/plain",
};

async function run({ chunkFilePath } : { chunkFilePath: string }) {
  const file = await uploadToGemini(chunkFilePath, "audio/x-m4a");

  const chatSession = gemini.startChat({
    generationConfig,
    history: [
      {
        role: "user",
        parts: [
          {
            fileData: {
              mimeType: file.mimeType,
              fileUri: file.uri,
            },
          },
        ],
      },
    ],
  });
  const result = await chatSession.sendMessage(
    "transcribe this audio, include timestamps and speaker diarization. this audio clip is from the podcast called 'My First Million'. speakers are Sam Parr and Shaan Puri"
  );
  console.log(result.response.text());
}

await run({ chunkFilePath: "../data/episode1/pod_chunk1.m4a" });


Uploaded file ../data/episode1/pod_chunk1.m4a as: files/kzzy4gwnf4dd
Okay, here's the transcription of the audio clip from "My First Million", with timestamps and speaker diarization:

**[00:00:00] Speaker: Sam Parr**
Screw Nick Huber talking about RV parks and storage units. We're going to be talking about flea markets. That's what I think is going to happen.

**[00:00:11] Speaker: Shaan Puri**
I want to talk to you about a topic and I think this is a topic it's my passion this topic. But it's also I'm going to bring it up because I think it's a trend and that's what we're about. And B, I ha it's sort of a prediction. I I I think I have a prediction here and it's going to be like be interesting in the next five and 10 years, okay?

**[00:00:42] Speaker: Sam Parr**
Sweaters, denim, tobacco.

**[00:00:47] Speaker: Shaan Puri**
Yeah, you're in the ballpark. Yeah. We're in we're in the zip code. We're definitely in the zip code.

**[00:00:53] Speaker: Sam Parr**
Media. Newspaper industry.